In [1]:
import numpy as np

In [39]:
class RecurrentNeuralNetwork:
    def __init__(self, inputX, outputY, reccLen, expOutput, lr):
        self.x = np.zeros(inputX)
        self.inputX = inputX
        self.y = np.zeros(outputY)
        self.outputY = outputY
        self.w = np.random.random((outputY, outputY))
        self.G = np.zeros_like(self.w)
        self.reccLen = reccLen
        self.lr = lr
        
        self.inputArr = np.zeros((reccLen+1, inputX))
        self.cellStatesArr = np.zeros((reccLen+1, outputY))
        self.outputArr = np.zeros((reccLen+1, outputY))
        self.hiddenStatesArr = np.zeros((reccLen+1, outputY))
        
        self.inputGate = np.zeros((reccLen+1, outputY))
        self.cellState = np.zeros((reccLen+1, outputY))
        self.outputGate = np.zeros((reccLen+1, outputY))
        self.forgetGate = np.zeros((reccLen+1, outputY))
        
        self.expOutput = np.vstack((np.zeros(expOutput.shape[0]), expOutput.T))
        self.LSTM = LSTM(inputX, outputY, reccLen, lr)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def derivativeSigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def forwardProp(self):
        for i in range(1, self.reccLen+1):
            self.LSTM.x = np.hstack((self.hiddenStatesArr[i-1], self.x))
            cs, hs, fgate, igate, cunit, ogate = self.LSTM.forwardProp()
            self.cellStatesArr[i]  = cs
            self.hiddenStatesArr[i] = hs
            self.forgetGate[i] = fgate
            self.inputArr[i] = igate
            self.cellState[i] = cunit
            self.outputGate[i] = ogate
            self.outputArr[i] = self.sigmoid(np.dot(self.w, hs))
            self.x = self.expOutput[i-1]
        return self.outputArr
    
    def backProp(self):
        totalError = 0
        #RNN
        dfcs = np.zeros(self.outputY)
        dfhs = np.zeros(self.outputY)
        weigths = np.zeros((self.outputY, self.outputY))
        
        #LSTM
        wfgate = np.zeros((self.outputY, self.inputX+self.outputY))
        wigate = np.zeros((self.outputY, self.inputX+self.outputY))
        wcunit = np.zeros((self.outputY, self.inputX+self.outputY))
        wogate = np.zeros((self.outputY, self.inputX+self.outputY))
        
        #loop backwards
        for i in range(self.reccLen, -1, -1):
            error = self.outputArr[i] - self.expOutput[i]
            weigths += np.dot(
                np.atleast_2d(error * self.derivativeSigmoid(self.outputArr[i])),
                np.atleast_2d(self.hiddenStatesArr[i]).T)
            #error * RNN weight matrix
            error = np.dot(error, self.w)
            #set input values to the LSTM cell for recurrence i
            self.LSTM.x = np.hstack((self.hiddenStatesArr[i], self.inputArr[i]))
            #set cell state of LSTM cell for recurrence i
            self.LSTM.cellState = self.cellStatesArr[i]
            
            #finally call the LSTM cell's backprop and retrive the gradients
            fgate, igate, cunit, ogate, dfcs, dfhs = self.LSTM.backProp(
                error, 
                self.cellStatesArr[i-1],
                self.forgetGate[i],
                self.inputGate[i],
                self.cellState[i],
                self.outputGate[i],
                dfcs, dfhs)
            
            #calculate total error
            totalError += np.sum(error)
            
            #accumulate all the gradients
            wfgate += fgate
            wigate += igate
            wcunit += cunit
            wogate += ogate
            
        #update LSTM matrices with average of accumulated gradient updates
        self.LSTM.update(wfgate/self.reccLen,
                         wigate/self.reccLen, 
                         wcunit/self.reccLen, 
                         wogate/self.reccLen)
        self.update(weigths/self.reccLen)
        return totalError
    
    
    def update(self, u):
        #vanilla implementation of RMSProp
        self.G = 0.9 * self.G + 0.1 * u**2
        self.w -= self.lr / np.sqrt(self.G + 1e-8) * u
        return
    
    
    #This is where we generate some sample text after gaving fully trained our model
    def sample(self):
        for i in range(1, self.reccLen + 1):
            self.LSTM.x = np.hstack((self.hiddenStatesArr[i-1], self.x))
            cs, hs, fg, inp, cu, og = self.LSTM.forwardProp()
            maxVec = np.argmax(self.x)
            self.x = np.zeros_like(self.x)
            self.x[maxVec] = 1
            self.inputArr[i] = self.x
            self.cellStatesArr[i] = cs
            self.hiddenStatesArr[i]= hs
            self.forgetGate[i] = fg
            self.inputGate[i] = inp
            self.cellState[i] = cu
            self.outputGate[i] = og
            self.outputArr[i] = self.sigmoid(np.dot(self.w, hs))
            maxVec = np.argmax(self.outputArr[i])
            newX = np.zeros_like(self.x)
            newX[maxVec] = 1
            self.x = newX
        return self.outputArr            

In [40]:
class LSTM:
    def __init__(self, xs, ys, reclen, lr):
        self.x = np.zeros(xs+ys)
        self.xs = xs + ys
        self.y = np.zeros(ys)
        self.ys = ys
        self.cs = np.zeros(ys)
        self.reclen = reclen
        self.lr = lr
        self.fgate = np.random.random((ys, xs+ys))
        self.igate = np.random.random((ys, xs+ys))
        self.cunit = np.random.random((ys, xs+ys))
        self.ogate = np.random.random((ys, xs+ys))
        self.fgradient = np.zeros_like(self.fgate)
        self.igradient = np.zeros_like(self.igate)
        self.cgradient = np.zeros_like(self.cunit)
        self.ogradient = np.zeros_like(self.ogate)
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def derivativeSigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def tanget(self, x):
        return np.tanh(x)
    
    def derivativeTanget(self, x):
        return 1 - np.tanh(x)**2
    
    def forwardProp(self):
        fgate = self.sigmoid(np.dot(self.fgate, self.x))
        self.cs *= fgate
        igate = self.sigmoid(np.dot(self.igate, self.x))
        cunit = self.sigmoid(np.dot(self.cunit, self.x))
        self.cs += igate * cunit
        ogate = self.sigmoid(np.dot(self.ogate, self.x))
        self.y = ogate * self.tanget(self.cs)
        return self.cs, self.y, fgate, igate, cunit, ogate
    
    def backProp(self, error, pcs, fgate, igate, cunit, ogate, dfcs, dfhs):
        error = np.clip(error + dfhs, -6, 6)
        df_outputgate = self.tanget(self.cs) * error
        outputgate_update = np.dot(np.atleast_2d(df_outputgate * self.derivativeTanget(ogate)).T, np.atleast_2d(self.x))
        
        df_cellstate = np.clip(error * ogate * self.derivativeTanget(self.cs) + dfcs, -6, 6)
        
        df_cellunit = df_cellstate * igate
        cellunit_update = np.dot(np.atleast_2d(df_cellunit * self.derivativeTanget(cunit)).T, np.atleast_2d(self.x))
        
        df_inputgate  = df_cellstate * cunit
        inputgate_update = np.dot(np.atleast_2d(df_inputgate * self.derivativeTanget(igate)).T, np.atleast_2d(self.x))
        
        df_forgetgate = df_cellstate * fgate
        forgetgate_update = np.dot(np.atleast_2d(df_forgetgate * self.derivativeTanget(fgate)).T, np.atleast_2d(self.x))
        
        dpcs = df_cellstate * fgate
        
        dphs = np.dot(df_cellunit, self.cunit)[:self.ys] + np.dot(df_outputgate, self.ogate)[:self.ys] + np.dot(df_inputgate, self.igate)[:self.ys] + np.dot(df_forgetgate, self.fgate)[:self.ys]
        
        return forgetgate_update, inputgate_update, cellunit_update, outputgate_update, dpcs, dphs
    
    
    def update(self, fgate, igate, cunit, ogate):
        self.fgradient = 0.9 * self.fgradient + 0.1 * fgate**2
        self.igradient = 0.9 * self.igradient + 0.1 * igate**2
        self.cgradient = 0.9 * self.cgradient + 0.1 * cunit**2
        self.ogradient = 0.9 * self.ogradient + 0.1 * ogate**2

In [45]:
def LoadText():
    with open('sample.txt', 'r') as textfile:
        data = textfile.read()
    text = list(data)
    outputSize = len(text)
    data = list(set(text))
    uniqueWords, dataSize = len(data), len(data) 
    returnData = np.zeros((uniqueWords, dataSize))
    for i in range(0, dataSize):
        returnData[i][i] = 1
    returnData = np.append(returnData, np.atleast_2d(data), axis=0)
    output = np.zeros((uniqueWords, outputSize))
    for i in range(0, outputSize):
        index = np.where(np.asarray(data) == text[i])
        output[:,i] = returnData[0:-1,index[0]].astype(float).ravel()  
    return returnData, uniqueWords, output, outputSize, data

#write the predicted output (series of words) to disk
def ExportText(output, data):
    finalOutput = np.zeros_like(output)
    prob = np.zeros_like(output[0])
    outputText = ""
    print(len(data))
    print(output.shape[0])
    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            prob[j] = output[i][j] / np.sum(output[i])
        outputText += np.random.choice(data, p=prob)    
    with open("output.txt", "w") as text_file:
        text_file.write(outputText)
    return

In [51]:
#Begin program    
print("Beginning")
iterations = 5000
learningRate = 0.001
#load input output data (words)
returnData, numCategories, expectedOutput, outputSize, data = LoadText()
print("Done Reading")
#init our RNN using our hyperparams and dataset
RNN = RecurrentNeuralNetwork(numCategories, numCategories, outputSize, expectedOutput, learningRate)

#training time!
for i in range(1, iterations):
    #compute predicted next word
    RNN.forwardProp()
    #update all our weights using our error
    error = RNN.backProp()
    #once our error/loss is small enough
    print("Error on iteration ", i, ": ", error)
    if error > -100 and error < 100 or i % 100 == 0:
        #we can finally define a seed word
        seed = np.zeros_like(RNN.x)
        maxI = np.argmax(np.random.random(RNN.x.shape))
        seed[maxI] = 1
        RNN.x = seed  
        #and predict some new text!
        output = RNN.sample()
        print(output)    
        #write it all to disk
        ExportText(output, data)
        print("Done Writing")
print("Complete")

Beginning
Done Reading
Error on iteration  1 :  23.810694169292287
[[0.         0.         0.         0.        ]
 [0.80651959 0.75627794 0.6455149  0.76662187]
 [0.86492668 0.81112184 0.69001742 0.824423  ]
 [0.87716492 0.82488423 0.69844085 0.83679777]
 [0.880054   0.8281226  0.70070952 0.83978503]
 [0.88089713 0.82908984 0.70141963 0.84066589]
 [0.88120871 0.82945598 0.70170193 0.84099248]
 [0.88134966 0.82962519 0.70183861 0.84114004]]
4
8
Done Writing
Error on iteration  2 :  27.843769018835925
[[0.         0.         0.         0.        ]
 [0.81282316 0.76223253 0.64830694 0.77242973]
 [0.86844511 0.81481618 0.6916694  0.82776968]
 [0.87826819 0.82595592 0.6982823  0.83773155]
 [0.88000193 0.82788761 0.69960902 0.83951559]
 [0.88036063 0.82829742 0.6999061  0.83988605]
 [0.88046664 0.82842307 0.70000483 0.83999564]
 [0.88051398 0.82848094 0.70005351 0.84004454]]
4
8
Done Writing
Error on iteration  3 :  27.632117728395656
[[0.         0.         0.         0.        ]
 [0.812147

C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\snettani\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less


Done Writing
Error on iteration  45 :  23.658606576300134
[[0.         0.         0.         0.        ]
 [0.78875195 0.72437209 0.62763574 0.75132465]
 [0.84761797 0.78816712 0.65520851 0.80269796]
 [0.85886586 0.80019482 0.66177651 0.81340395]
 [0.86095485 0.80245808 0.66313835 0.81545591]
 [0.86144071 0.80300234 0.66348683 0.81593887]
 [0.86160285 0.80319101 0.6636184  0.81610013]
 [0.86167962 0.80328263 0.66368613 0.81617632]]
4
8
Done Writing
Error on iteration  46 :  23.574185612152544
[[0.         0.         0.         0.        ]
 [0.80387651 0.74803144 0.61716266 0.76535027]
 [0.84828006 0.78858566 0.65601413 0.80299932]
 [0.85909074 0.80041184 0.66157124 0.81349743]
 [0.86088353 0.8023204  0.66267774 0.81527322]
 [0.8612234  0.80269136 0.66289882 0.81561372]
 [0.86131055 0.80279034 0.6629638  0.81570123]
 [0.8613461  0.8028324  0.66299423 0.81573677]]
4
8
Done Writing
Error on iteration  47 :  23.499633673353426
[[0.         0.         0.         0.        ]
 [0.78794965 0.72

Error on iteration  89 :  20.20525574061047
[[0.         0.         0.         0.        ]
 [0.77890991 0.71830278 0.59254689 0.73264815]
 [0.82817924 0.76293436 0.62391699 0.77856787]
 [0.84023922 0.77582369 0.62822742 0.79017212]
 [0.84229115 0.77795172 0.62924953 0.79213056]
 [0.84273658 0.77843068 0.62949612 0.79255609]
 [0.84287584 0.77858687 0.62958594 0.79268839]
 [0.84293992 0.77866101 0.629632   0.79274893]]
4
8
Done Writing
Error on iteration  90 :  20.125141207519675
[[0.         0.         0.         0.        ]
 [0.76967041 0.7016698  0.60135065 0.73002325]
 [0.82815893 0.76323541 0.62213752 0.77900274]
 [0.83960783 0.77502825 0.62730233 0.78946601]
 [0.84173758 0.77724816 0.62838651 0.79147496]
 [0.8422307  0.77778066 0.62866543 0.79194498]
 [0.84239399 0.77796429 0.62877166 0.79210017]
 [0.8424708  0.77805308 0.62882663 0.79217282]]
4
8
Done Writing
Error on iteration  91 :  20.047478024771927
[[0.         0.         0.         0.        ]
 [0.77425374 0.71683732 0.59267

Done Writing
Error on iteration  133 :  16.92847883263685
[[0.         0.         0.         0.        ]
 [0.75987029 0.69608099 0.56641931 0.71111966]
 [0.80730694 0.73666026 0.5905399  0.75348043]
 [0.81947501 0.74919008 0.59327516 0.76473255]
 [0.82154928 0.7512491  0.59400973 0.7666245 ]
 [0.82199966 0.75171366 0.59418957 0.76703414]
 [0.8221398  0.75186482 0.59425641 0.76716017]
 [0.82220381 0.75193619 0.59429105 0.76721715]]
4
8
Done Writing
Error on iteration  134 :  16.857229060091427
[[0.         0.         0.         0.        ]
 [0.75509887 0.69472739 0.56672247 0.70671277]
 [0.8090223  0.73843515 0.58995416 0.75512785]
 [0.81940571 0.74901296 0.59267979 0.7645105 ]
 [0.8212315  0.75081508 0.59328986 0.76617919]
 [0.82159657 0.75118606 0.59342669 0.76651162]
 [0.82169923 0.75129528 0.59347286 0.76660396]
 [0.82174352 0.75134434 0.59349614 0.76664328]]
4
8
Done Writing
Error on iteration  135 :  16.786776427519033
[[0.         0.         0.         0.        ]
 [0.76595081 0.

...

Error on iteration  4863 :  -0.34216882166226426
[[0.         0.         0.         0.        ]
 [0.38622286 0.31307222 0.19837399 0.33366335]
 [0.34232075 0.25774241 0.15076323 0.27508122]
 [0.33692958 0.25050851 0.14009565 0.26661429]
 [0.33582943 0.24904239 0.13831576 0.26503918]
 [0.33557419 0.24873522 0.13796002 0.26470377]
 [0.33548165 0.24863833 0.13785898 0.26459437]
 [0.33543157 0.24859047 0.13781332 0.26453882]]
4
8
Done Writing
Error on iteration  4864 :  -0.3205819206450158
[[0.         0.         0.         0.        ]
 [0.38675704 0.31356089 0.19875228 0.33415061]
 [0.34296305 0.25828933 0.15113285 0.27563956]
 [0.33760449 0.25107539 0.14045861 0.26719915]
 [0.33651463 0.24961648 0.13868024 0.26563451]
 [0.33626387 0.24931297 0.13832659 0.26530415]
 [0.33617376 0.24921816 0.13822707 0.26519751]
 [0.33612514 0.24917159 0.13818244 0.26514358]]
4
8
Done Writing
Error on iteration  4865 :  -0.33789417442129593
[[0.         0.         0.         0.        ]
 [0.37539645 0.2972

[[0.         0.         0.         0.        ]
 [0.37932941 0.31087101 0.20589474 0.3232393 ]
 [0.34279915 0.25795138 0.15049949 0.27520675]
 [0.33730171 0.25080871 0.14032609 0.26696099]
 [0.33618307 0.24934746 0.13854034 0.26537805]
 [0.33588887 0.24900589 0.13815323 0.26499779]
 [0.33577057 0.24888483 0.13802908 0.26485847]
 [0.33570461 0.24882203 0.13796921 0.26478475]]
4
8
Done Writing
Error on iteration  4896 :  -0.3293340592824845
[[0.         0.         0.         0.        ]
 [0.37562446 0.29748057 0.21357849 0.32741694]
 [0.344717   0.26028884 0.15234015 0.27785242]
 [0.33831133 0.25176575 0.14118208 0.2680677 ]
 [0.33680792 0.24991496 0.13901653 0.26601498]
 [0.33634563 0.24940951 0.13846982 0.26544476]
 [0.33614154 0.24920726 0.13826898 0.26521093]
 [0.3360278  0.24909987 0.13816768 0.26508507]]
4
8
Done Writing
Error on iteration  4897 :  -0.34034658137973406
[[0.         0.         0.         0.        ]
 [0.37899512 0.31056683 0.2056625  0.32292874]
 [0.34238521 0.257599

Error on iteration  4922 :  -0.32079904839018053
[[0.         0.         0.         0.        ]
 [0.37575354 0.29759236 0.21365885 0.32752382]
 [0.34488419 0.26042761 0.1524215  0.27798662]
 [0.33849742 0.25191972 0.14127469 0.26822354]
 [0.33700269 0.2500771  0.13911687 0.2661819 ]
 [0.33654444 0.24957562 0.13857409 0.26561684]
 [0.33634233 0.24937531 0.13837516 0.26538547]
 [0.33622964 0.24926893 0.13827484 0.26526087]]
4
8
Done Writing
Error on iteration  4923 :  -0.34530809853122335
[[0.         0.         0.         0.        ]
 [0.37907664 0.31064039 0.20571699 0.32300198]
 [0.34248608 0.25768482 0.15032035 0.27492775]
 [0.33697782 0.25053616 0.14015064 0.26667666]
 [0.33585748 0.24907428 0.13836597 0.26509333]
 [0.33556303 0.24873271 0.13797922 0.26471317]
 [0.33544468 0.24861168 0.13785522 0.26457393]
 [0.3353787  0.2485489  0.13779542 0.26450024]]
4
8
Done Writing
Error on iteration  4924 :  -0.3211070751128116
[[0.         0.         0.         0.        ]
 [0.38669529 0.3135

Error on iteration  4947 :  -0.334224296483053
[[0.         0.         0.         0.        ]
 [0.37929346 0.31083767 0.20586753 0.32320346]
 [0.34275456 0.25791291 0.15047267 0.27516546]
 [0.33725604 0.25077001 0.14030074 0.26692022]
 [0.33613745 0.24930907 0.13851565 0.26533789]
 [0.33584336 0.24896769 0.13812881 0.26495793]
 [0.33572512 0.24884672 0.13800478 0.26481873]
 [0.33565919 0.24878396 0.13794496 0.26474507]]
4
8
Done Writing
Error on iteration  4948 :  -0.32818090834356495
[[0.         0.         0.         0.        ]
 [0.37561052 0.29746808 0.21356841 0.3274037 ]
 [0.34469969 0.26027403 0.15233016 0.2778369 ]
 [0.33829336 0.2517506  0.1411723  0.26805188]
 [0.33678985 0.24989978 0.13900683 0.26599915]
 [0.33632754 0.24939433 0.13846014 0.26542895]
 [0.33612344 0.24919208 0.13825931 0.26519512]
 [0.3360097  0.24908469 0.13815802 0.26506926]]
4
8
Done Writing
Error on iteration  4949 :  -0.33989120552002205
[[0.         0.         0.         0.        ]
 [0.39129402 0.31753

Done Writing
Error on iteration  4979 :  -0.32755887647737325
[[0.         0.         0.         0.        ]
 [0.37945302 0.31098337 0.20598033 0.32335393]
 [0.34295224 0.25808137 0.15058621 0.27534213]
 [0.33746043 0.25094209 0.14041168 0.26709987]
 [0.33634282 0.24948144 0.13862576 0.26551758]
 [0.33604885 0.24914    0.13823862 0.26513747]
 [0.33593061 0.24901899 0.13811447 0.26499819]
 [0.33586468 0.2489562  0.13805459 0.26492449]]
4
8
Done Writing
Error on iteration  4980 :  -0.33336099299126976
[[0.         0.         0.         0.        ]
 [0.38643259 0.31326796 0.19853532 0.33385186]
 [0.34257245 0.25795899 0.15091384 0.27529864]
 [0.33719461 0.25073193 0.14024009 0.26684323]
 [0.33609914 0.2492686  0.13845971 0.26527309]
 [0.33584615 0.24896307 0.13810459 0.26494024]
 [0.3357549  0.24886722 0.1380042  0.26483231]
 [0.33570562 0.24882004 0.13795904 0.26477764]]
4
8
Done Writing
Error on iteration  4981 :  -0.32745162213421336
[[0.         0.         0.         0.        ]
 [0.3